In [1]:
import sys
sys.path.append("/home/webphy/Desktop/dnn_processor/")  # just to enable `dataset`
sys.path.append("/home/webphy/Desktop/dnn_processor/dataset/")  # just to enable `dataset`

import tensorflow as tf
import dataset
import numpy as np
import keras
from datetime import datetime
from keras import layers

2024-02-11 11:20:27.140031: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-11 11:20:27.163223: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-11 11:20:27.163244: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-11 11:20:27.163941: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-11 11:20:27.167924: I tensorflow/core/platform/cpu_feature_guar

In [2]:
DEFAULT_BATCH_SIZE = 128
EPOCHS = 100
DEFAULT_INPUT_SIZE = 128

train_images_rgx = "../../dataset/train_set_4x/*/*.jpeg"
test_images_rgx = "../../dataset/test_set_4x/*/*.jpeg"

train_dataset, test_dataset, num_classes = dataset.create_datasets(
    train_images_rgx, test_images_rgx, DEFAULT_BATCH_SIZE, DEFAULT_INPUT_SIZE
)

2024-02-11 11:20:28.141297: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-11 11:20:28.165820: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-11 11:20:28.165923: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

# load the base line model

In [3]:
EXP_ID = "baseline_model"

baseline_model = keras.models.load_model(
    f"../handmade/saved_models/{EXP_ID}/best_model_t/model.h5",
    compile=False,
)

## ADD A LINEAR ACTIVATION LAYER BEFORE FLATTEN
x = x_in = keras.Input(shape=(128,128,3))

x = keras.layers.Activation("linear", name="qtz_inputs")(x)

for l in baseline_model.layers[1:]:
    if isinstance(l, keras.layers.Flatten):
        x = keras.layers.Activation("linear", name="qtz_flatten_input")(x)
    
    x = l(x)

model = keras.Model(x_in, x)

model.compile(
    keras.optimizers.legacy.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(True), metrics=["accuracy"]
)

model.save("tempmodel.h5")


model = keras.models.load_model("tempmodel.h5", compile=False)
model.compile(
    keras.optimizers.legacy.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(True), metrics=["accuracy"]
)


score = model.evaluate(test_dataset)
print("Accuracy: {}".format(score[1]))

model.summary()
baseline_model.summary()

/home/webphy/Desktop/dnn_processor_v2/.venv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
2024-02-11 11:20:30.377967: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


3/3 [==============================] - 2s 41ms/step - loss: 0.4989 - accuracy: 0.8582
Accuracy: 0.8582375645637512
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 qtz_inputs (Activation)     (None, 128, 128, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 4)         196       
                                                                 
 activation (Activation)     (None, 32, 32, 4)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 15, 15, 16)        592       
                                                                 
 activation_1 (Activation)   (None, 15, 15, 16)        0         
            


# Quantizing a Model With AutoQKeras

To quantize this model with AutoQKeras, we need to define the quantization for kernels, biases and activations; forgiving factors and quantization strategy.

Below we define which quantizers are allowed for kernel, bias, activations and linear. Linear is a proxy that we use to capture Activation("linear") to apply quantization without applying a non-linear operation. In some networks, we found that this trick may be necessary to better represent the quantization space.


In [4]:
# These are the quantizers we'll test in the bayesian optimization
quantization_config = {
    "kernel": {
        "quantized_bits(2,0,1,alpha=1.0)": 2,
        "quantized_bits(4,0,1,alpha=1.0)": 4,
        "quantized_bits(6,0,1,alpha=1.0)": 6,
        "quantized_bits(8,0,1,alpha=1.0)": 8,
    },
    "bias": {
        "quantized_bits(2,0,1,alpha=1.0)": 2,
        "quantized_bits(4,0,1,alpha=1.0)": 4,
        "quantized_bits(6,0,1,alpha=1.0)": 6,
        "quantized_bits(8,0,1,alpha=1.0)": 8,
    },
    "activation": {
        "quantized_relu(3,1)": 3,
        "quantized_relu(4,2)": 4,
        "quantized_relu(8,2)": 8,
        "quantized_relu(8,4)": 8,
        "quantized_relu(16,6)": 16,
    },
    "linear": {
        "quantized_bits(2,0,1,alpha=1.0)": 2,
        "quantized_bits(4,0,1,alpha=1.0)": 4,
        "quantized_bits(6,0,1,alpha=1.0)": 6,
        "quantized_bits(8,0,1,alpha=1.0)": 8,
    },
}

# These are the layer types we will quantize
limit = {
    "Dense": [8, 8, 8],
    "Conv2D": [8, 8, 8],
    "Activation": [8],
}

# Use this if you want to minimize the model bit size
goal_bits = {
    "type": "bits",
    "params": {
        "delta_p": 5.0,  # We tolerate up to a +5% accuracy change
        "delta_n": 5.0,  # We tolerate down to a -5% accuracy change
        "rate": 2.0,  # We want a x2 times smaller model
        "stress": 1.0,  # Force the reference model size to be smaller by setting stress<1
        "input_bits": 8,
        "output_bits": 8,
        "ref_bits": 8,
        "config": {"default": ["parameters", "activations"]},
    },
}

# Use this if you want to minimize the model energy consumption
goal_energy = {
    "type": "energy",
    "params": {
        "delta_p": 5.0,
        "delta_n": 5.0,
        "rate": 1.1,
        "stress": 1.0,
        "process": "horowitz",
        "parameters_on_memory": ["sram", "sram"],
        "activations_on_memory": ["sram", "sram"],
        "rd_wr_on_io": [False, False],
        "min_sram_size": [0, 0],
        "source_quantizers": ["fp32"],
        "reference_internal": "int8",
        "reference_accumulator": "int32",
    },
}

run_config = {    
    "goal": goal_bits,
    "quantization_config": quantization_config,
    "learning_rate_optimizer": False,
    "transfer_weights": True,  # Randomely initialize weights
    "mode": "bayesian",  # This can be bayesian,random,hyperband
    "seed": 42,
    "limit": limit,
    "tune_filters": "none",
    "tune_filters_exceptions": "^dense|^conv2d_2",
    "distribution_strategy": None,
    "max_trials": 200,  # Let's just do 5 trials for this demonstrator, ideally you should do as many as possible
}


In [5]:
import qkeras
from qkeras.autoqkeras.utils import print_qmodel_summary
from qkeras.autoqkeras import AutoQKeras
import time
n_epochs = 100

In [6]:
autoqk = AutoQKeras(
    model, output_dir="autoqqatwqkeras", metrics=["acc"], custom_objects={}, **run_config
)
autoqk.fit(train_dataset, validation_data=test_dataset, epochs=30)

Trial 200 Complete [00h 00m 06s]
val_score: 1.5852938890457153

Best val_score So Far: 1.872563123703003
Total elapsed time: 00h 27m 14s


In [8]:
aqmodel = autoqk.get_best_model()
aqmodel.save_weights("autoqqatwqkeras/bestmodel/model_weights.h5")
print_qmodel_summary(aqmodel)

learning_rate: 0.0010000000474974513
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 qtz_inputs (QActivation)    (None, 128, 128, 3)       0         
                                                                 
 conv2d (QConv2D)            (None, 32, 32, 4)         196       
                                                                 
 activation (QActivation)    (None, 32, 32, 4)         0         
                                                                 
 conv2d_1 (QConv2D)          (None, 15, 15, 16)        592       
                                                                 
 activation_1 (QActivation)  (None, 15, 15, 16)        0         
                                                                 
 conv2d_2 (QConv2D)     

In [9]:
layers = [l for l in aqmodel.layers]
x = layers[0].output
for i in range(1, len(layers)):
    x = layers[i](x)

new_model = keras.Model(inputs=[layers[0].input], outputs=[x])
LOSS = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=0.0005)

new_model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=["accuracy"])
new_model.summary()
new_model.load_weights("autoqqatwqkeras/bestmodel/model_weights.h5")
new_model.save("autoqqatwqkeras/bestmodel/model.h5")
print_qmodel_summary(new_model)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 qtz_inputs (QActivation)    (None, 128, 128, 3)       0         
                                                                 
 conv2d (QConv2D)            (None, 32, 32, 4)         196       
                                                                 
 activation (QActivation)    (None, 32, 32, 4)         0         
                                                                 
 conv2d_1 (QConv2D)          (None, 15, 15, 16)        592       
                                                                 
 activation_1 (QActivation)  (None, 15, 15, 16)        0         
                                                                 
 conv2d_2 (QConv2D)          (None, 15, 15, 5)         85    

/home/webphy/Desktop/dnn_processor_v2/.venv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
/home/webphy/Desktop/dnn_processor_v2/.venv/lib/python3.10/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(


In [10]:
model.evaluate(test_dataset)
new_model.evaluate(test_dataset)

3/3 [==============================] - 0s 5ms/step - loss: 1.3269 - accuracy: 0.6820


[1.3268662691116333, 0.6819923520088196]

In [11]:
new_model.fit(train_dataset, batch_size=256, validation_data=test_dataset, epochs=10)
new_model.evaluate(test_dataset)

Epoch 1/10


2024-02-11 11:48:37.218867: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape ingradient_tape/model/activation_1/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-02-11 11:48:37.309912: I external/local_xla/xla/service/service.cc:168] XLA service 0x55c21f33d940 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-11 11:48:37.309986: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2024-02-11 11:48:37.319980: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1707662917.388216 1223502 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


27/27 [==============================] - 1s 9ms/step - loss: 0.7650 - accuracy: 0.7896 - val_loss: 0.9767 - val_accuracy: 0.7625
Epoch 2/10
27/27 [==============================] - 0s 10ms/step - loss: 0.4795 - accuracy: 0.8359 - val_loss: 1.0554 - val_accuracy: 0.7739
Epoch 3/10
27/27 [==============================] - 0s 10ms/step - loss: 0.4506 - accuracy: 0.8513 - val_loss: 1.0043 - val_accuracy: 0.7548
Epoch 4/10
27/27 [==============================] - 0s 10ms/step - loss: 0.5191 - accuracy: 0.8368 - val_loss: 1.0604 - val_accuracy: 0.7548
Epoch 5/10
27/27 [==============================] - 0s 10ms/step - loss: 0.3986 - accuracy: 0.8744 - val_loss: 0.5813 - val_accuracy: 0.8352
Epoch 6/10
27/27 [==============================] - 0s 10ms/step - loss: 0.3709 - accuracy: 0.8744 - val_loss: 0.6404 - val_accuracy: 0.8084
Epoch 7/10
27/27 [==============================] - 0s 10ms/step - loss: 0.4018 - accuracy: 0.8666 - val_loss: 0.8224 - val_accuracy: 0.8123
Epoch 8/10
27/27 [=======

[0.6470230221748352, 0.8160919547080994]

In [12]:
print_qmodel_summary(new_model)

qtz_inputs           quantized_bits(4,0,1,alpha=1.0)
conv2d               f=4 quantized_bits(4,0,1,alpha=1.0) quantized_bits(4,0,1,alpha=1.0) 
activation           quantized_relu(3,1)
conv2d_1             f=16 quantized_bits(6,0,1,alpha=1.0) quantized_bits(8,0,1,alpha=1.0) 
activation_1         quantized_relu(3,1)
conv2d_2             f=5 quantized_bits(4,0,1,alpha=1.0) quantized_bits(2,0,1,alpha=1.0) 
qtz_flatten_input    quantized_bits(8,0,1,alpha=1.0)
dense                u=5 quantized_bits(4,0,1,alpha=1.0) quantized_bits(2,0,1,alpha=1.0) 



In [13]:
from qkeras.estimate import print_qstats

print_qstats(new_model)


Number of operations in model:
    conv2d                        : 196608 (smult_4_4)
    conv2d_1                      : 129600 (smult_6_3)
    conv2d_2                      : 18000 (smult_4_3)
    dense                         : 5625  (smult_4_8)

Number of operation types in model:
    smult_4_3                     : 18000
    smult_4_4                     : 196608
    smult_4_8                     : 5625
    smult_6_3                     : 129600

Weight profiling:
    conv2d_weights                 : 192   (4-bit unit)
    conv2d_bias                    : 4     (4-bit unit)
    conv2d_1_weights               : 576   (6-bit unit)
    conv2d_1_bias                  : 16    (8-bit unit)
    conv2d_2_weights               : 80    (4-bit unit)
    conv2d_2_bias                  : 5     (2-bit unit)
    dense_weights                  : 5625  (4-bit unit)
    dense_bias                     : 5     (2-bit unit)

Weight sparsity:
... quantizing model
    conv2d                         : 0